# Tested for `networks_2022`

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork

In [2]:
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'

In [3]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [36]:
y_hats = []

with h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r') as f_in:
    for ind in range(len(f_in['data'])):
        word, genre = net(torch.tensor(np.reshape( f_in['data'][ind],(164,400))))
        
        y_hat = int(torch.argmax(word))
        y_hats.append(y_hat)
        if len(y_hats) > 20:
            break

In [10]:
# Torch network outputs
print(y_hats[:20])

[229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46]


# Compare to TF labels

In [6]:
tf_y_hats = np.array([229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46, 152, 388, 278, 459, 256, 23, 14, 25, 26, 27, 129, 29, 30, 31, 63, 337, 34, 216, 36, 37, 38, 241, 40, 41, 183, 42, 393, 240, 45, 196, 46, 487, 48, 129, 142, 51, 269, 519, 54, 55, 284, 57, 65, 58, 239, 59, 376, 62, 63, 158, 345, 66, 67, 68, 69, 70, 394, 72, 494, 74, 109, 75, 48, 78, 343, 455, 487, 413, 82, 109, 84, 496, 243, 490, 9, 24, 89, 0, 110, 259])
tf_y_hats_no_Dropout = [229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46, 152, 388, 278, 459, 256, 23, 14, 25, 26, 27, 129, 29, 30, 31, 63, 337, 34, 216, 36, 37, 38, 241, 40, 41, 183, 42, 393, 240, 45, 196, 46, 487, 48, 129, 142, 51, 269, 519, 54, 55, 284, 57, 65, 58, 239, 59, 376, 62, 63, 158, 345, 66, 67, 68, 69, 70, 394, 72, 494, 74, 109, 75, 48, 78, 343, 455, 487, 413, 82, 109, 84, 496, 243, 490, 9, 24, 89, 0, 110, 259]
print(tf_y_hats[:20])

In [8]:
print(len(np.where(tf_y_hats == np.array(y_hats))[0]))

100


In [8]:
# http://localhost:1339/notebooks/JupyterNotebooks/thesis/analysesUsingNAC/net7_tf_2020w-naturalistic_heldOutSpkr.ipynb

# Think this is WSJ only -- 531 words in train set rather tha 587

# Compare to True labels

In [4]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [5]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [6]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

In [12]:
y_hats = []
corrects = []

with h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r') as f_in:
    for ind in range(len(f_in['data'])):
        word, genre = net(torch.tensor(np.reshape( f_in['data'][ind],(164,400))))
        
        y_hat = int(torch.argmax(word))
        y_hats.append(y_hat)
        corrects.append(y_hat == labels[ind])

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [17]:
import pickle

In [18]:
with open(f"{engram_dir}PsychophysicsWord2017W_net_performance.p", 'wb') as f:
    results = {
        'net_pred': np.array(y_hats),
        'net_mistakes': np.logical_not(corrects)
    }
    pickle.dump(results, f)